In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
from decodercell import DecoderCell

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
input_vocab_size = 30000
output_vocab_size = 30005
input_ph = tf.placeholder(dtype=tf.float32, shape=[64, None, input_vocab_size])
output_ph = tf.placeholder(dtype=tf.float32, shape=[64, None, output_vocab_size])

In [3]:
batch_size = tf.cast(tf.shape(input_ph)[0], tf.int32)
seq_length = tf.cast(tf.shape(input_ph)[1], tf.int32)

In [4]:
input_embedding_size = 650
W_input_embedding = tf.Variable(tf.random_normal([input_vocab_size, input_embedding_size], stddev=0.001))
b_input_embedding = tf.Variable(tf.random_normal([input_embedding_size], stddev=0.001))

In [5]:
encoder_gru_size = 1024
encoder_gru_fw = tf.nn.rnn_cell.GRUCell(encoder_gru_size)
encoder_gru_bw = tf.nn.rnn_cell.GRUCell(encoder_gru_size)

In [6]:
decoder_gru_size = 1152
#decoder_gru_tuple = tf.nn.rnn_cell.GRUCell(decoder_gru_size)

In [7]:
input_2d = tf.reshape(input_ph, [-1, input_vocab_size])
embedded_input_2d = tf.nn.relu(tf.matmul(input_2d, W_input_embedding) + b_input_embedding)
embedded_input_3d = tf.reshape(embedded_input_2d, [batch_size, seq_length, input_vocab_size])

In [8]:
# [h_fw, h_bw]
bidirect_encoder_states, _ = tf.nn.bidirectional_dynamic_rnn(encoder_gru_fw, encoder_gru_bw, embedded_input_3d, dtype=tf.float32)
#help(tf.nn.bidirectional_dynamic_rnn)

In [9]:
print(tf.shape(bidirect_encoder_states[0]))
print(tf.shape(bidirect_encoder_states[1]))

Tensor("Shape_2:0", shape=(3,), dtype=int32)
Tensor("Shape_3:0", shape=(3,), dtype=int32)


In [10]:
output_embedding_size = input_embedding_size

In [11]:
print(tf.__version__)
print(bidirect_encoder_states)

1.9.0-rc0
(<tf.Tensor 'bidirectional_rnn/fw/fw/transpose_1:0' shape=(?, ?, 1024) dtype=float32>, <tf.Tensor 'ReverseV2:0' shape=(?, ?, 1024) dtype=float32>)


In [12]:
thing = DecoderCell(encoder_gru_size*2, decoder_gru_size, tf.concat(bidirect_encoder_states, axis=-1))

encoder output:  Tensor("concat:0", shape=(?, ?, 2048), dtype=float32)


In [13]:
thing_output, _ = tf.nn.dynamic_rnn(thing, embedded_input_3d, dtype=tf.float32)

attention_d:  Tensor("rnn/while/decoder_cell/Tanh:0", shape=(?, ?, 512), dtype=float32)
attention_e:  Tensor("rnn/while/decoder_cell/einsum/transpose_2:0", shape=(?, ?), dtype=float32)
gru state size:  1152
gru output size:  1152
array_ops.concat([context, y_prev]):  Tensor("rnn/while/decoder_cell/concat:0", shape=(?, 2668), dtype=float32)
state_true:  Tensor("rnn/while/Identity_3:0", shape=(?, 1152), dtype=float32)
gru state:  Tensor("rnn/while/decoder_cell/gru_cell/add:0", shape=(?, 1152), dtype=float32)


In [14]:
a = np.random.rand(30002)

In [15]:
a.shape

(30002,)

In [17]:
b = a/np.sum(a)

In [18]:
np.sum(b)

1.0

In [19]:
b

array([3.05682386e-05, 5.10167527e-05, 5.90540277e-05, ...,
       3.35547923e-05, 4.23000651e-05, 4.48985801e-05])

In [20]:
b.argmax()

23687

In [21]:
b[b.argmax()]

6.674773254680515e-05

In [25]:
c = np.exp(100*a)/np.sum(np.exp(100*a))

In [26]:
c.argmax()

23687

In [27]:
c[c.argmax()]

0.003246000642831474

In [28]:
c[c.argmin()]

1.2147140872837425e-46

In [29]:
tf.initializers.orthogonal

tensorflow.python.ops.init_ops.Orthogonal

In [30]:
help(tf.initializers.orthogonal)

Help on class Orthogonal in module tensorflow.python.ops.init_ops:

class Orthogonal(Initializer)
 |  Initializer that generates an orthogonal matrix.
 |  
 |  If the shape of the tensor to initialize is two-dimensional, it is initialized
 |  with an orthogonal matrix obtained from the QR decomposition of a matrix of
 |  random numbers drawn from a normal distribution.
 |  If the matrix has fewer rows than columns then the output will have orthogonal
 |  rows. Otherwise, the output will have orthogonal columns.
 |  
 |  If the shape of the tensor to initialize is more than two-dimensional,
 |  a matrix of shape `(shape[0] * ... * shape[n - 2], shape[n - 1])`
 |  is initialized, where `n` is the length of the shape vector.
 |  The matrix is subsequently reshaped to give a tensor of the desired shape.
 |  
 |  Args:
 |    gain: multiplicative factor to apply to the orthogonal matrix
 |    seed: A Python integer. Used to create random seeds. See
 |      @{tf.set_random_seed}
 |      for b